In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
pd.set_option("display.max_colwidth", 510)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV

In [4]:
from sklearn.metrics import roc_auc_score, roc_curve
from nltk.stem.snowball import EnglishStemmer

recommender,   
user lists ONE ingredient   
returns cocktail with filter on ingredient.
vecto user input.  
+ returns 5 cocktail from different clusters (10-15 cocktail par cluster, cluster_id, 100+ clusters) 
filter on user ingredient, drop duplicates from clusters (select one only)  
from this cocktail, find other cocktails containing a common ingredient (Kmeans)

_______________________________

AI generated cocktails:  
table avec index = ingrdeient, col= cocktail name
(PCA)
user input = ingredient  
algo returns other ingredients that "go together" (Knearest neighbors in the table), those seen often together in cocktails.  
  
user can re-input a suggested ingredient, iterates on the process (sum or avg)  
stores the avg/count of the inputted ingredients, and reruns 

For one-hot encoded or count-encoded data (e.g. country, region, or text), you should normalize by row (i.e. individual), before concatenating with the other columns. You can use for that https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html

In [5]:
# ingredient_table # pour chaque cocktail, concat tt les ingredients DU cocktail (' ing1 ing2 ing3 ing4')
# #  ensuite utiliser countvextorizer qui va découper TOUS les mots (genre 'orange =1', 'juice=1')

In [6]:
# #  concat tables to have a table with 
# index = ctl_name
# col = ingredients (words)

In [7]:
ctl_name = pd.read_csv('./CSVs/ctl_name_id.csv', sep=',', index_col=[0])
ctl_ingredient_words = pd.read_csv('./CSVs/ingredient_table.csv', sep=',',index_col=[0])
ctl_ing_id = pd.read_csv('./CSVs/cocktail_ingredient_table.csv', sep=',')

In [8]:
ctl_ing_id.drop(columns='typing.Literal[<no_default>]', inplace=True)

In [9]:
ctl_ing_id

,cocktail_id,ingredient_id,quantity
0,0,461,1 oz
1,0,265,.5 oz
2,0,263,.25 oz
3,0,257,1 oz
4,0,523,1 tsp
...,...,...,...
18690,4652,965,3 oz
18691,4652,381,1 oz
18692,4675,88,3 oz
18693,4675,787,2 oz


In [10]:
ctl_ingredient_words.dropna(inplace=True)

In [11]:
ctl_ingredient_words

,ingredient_id,ingredients
1,1,Iced Tea
2,2,Italian Amaro
3,3,Simple Syrup
4,4,Triple Sec
5,5,Scotch Liqueur
...,...,...
982,982,Peppermint and Hot Cinnamon Schnapps
983,983,(1 ml) Vodka
984,984,Mixed Fruit
985,985,Mint Schnapps


In [12]:
ctl_name

,cocktail_id,name
0,0,American Beauty Cocktail
1,1,Azzuro
2,2,Apple Fairy
3,3,Black Rose Bacardi
4,4,Blue Latvian
...,...,...
4685,5111,Brown Hen
4686,5112,Bosom Caresser
4687,5113,Brandy Water Press
4688,5114,Big Dipper


In [13]:
df_temp_merge = pd.merge(ctl_ing_id,ctl_ingredient_words, on='ingredient_id')

In [14]:
df_temp_merge.drop(columns='quantity', inplace=True)

In [15]:
df_merged = pd.merge(df_temp_merge,ctl_name,on='cocktail_id' )

In [16]:
df_merged

,cocktail_id,ingredient_id,ingredients,name
0,0,461,Brandy,American Beauty Cocktail
1,0,265,Dry Vermouth,American Beauty Cocktail
2,0,263,White Creme De Menthe,American Beauty Cocktail
3,0,257,Orange Juice,American Beauty Cocktail
4,0,523,Grenadine Syrup,American Beauty Cocktail
...,...,...,...,...
18677,4430,475,(4500 ml) Watermelon,Vodka infused Watermelon
18678,4430,983,(1 ml) Vodka,Vodka infused Watermelon
18679,4675,88,(90 ml) Prosecco,Aperol Spritz
18680,4675,787,(60 ml) Aperol,Aperol Spritz


__________________________________

## Create table to tokenize

In [17]:
df_ing_ctl_matrix = df_merged.drop(columns=['cocktail_id','ingredient_id'])

In [18]:
df_ing_ctl_matrix

,ingredients,name
0,Brandy,American Beauty Cocktail
1,Dry Vermouth,American Beauty Cocktail
2,White Creme De Menthe,American Beauty Cocktail
3,Orange Juice,American Beauty Cocktail
4,Grenadine Syrup,American Beauty Cocktail
...,...,...
18677,(4500 ml) Watermelon,Vodka infused Watermelon
18678,(1 ml) Vodka,Vodka infused Watermelon
18679,(90 ml) Prosecco,Aperol Spritz
18680,(60 ml) Aperol,Aperol Spritz


In [19]:
df_ctl_info = pd.read_csv('./CSVs/cocktail_info.csv', sep=',')

In [20]:
df_ctl_info = df_ctl_info[['Name','Ingredients']]

In [21]:
df_ctl_info.head()

,Name,Ingredients
0,American Beauty Cocktail,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz: White Creme De Menthe | 1 oz: Orange Juice | 1 tsp: Grenadine Syrup | 1 oz: Red Port
1,Azzuro,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg
2,Apple Fairy,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodka | 3 oz: Apple Soda
3,Black Rose Bacardi,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz: Lime Juice
4,Blue Latvian,| 1 oz: Bourbon Whiskey |: Raspberry Juice |: Lemon-Lime Soda


In [22]:
df_ctl_info.loc[:,'Ingredients']= df_ctl_info.loc[:,'Ingredients'].str.split('|')

In [23]:
df_ctl_info.loc[:,'Ingredients'] = df_ctl_info.loc[:,'Ingredients'].apply(lambda x: ' '.join(x))

In [24]:
import re

def use_regex(input_text):
    pattern = re.sub(r'\b\d+(\.\d+)?\s*\w+\b|[^\w\s]', '', string = input_text)
    return pattern

In [25]:
df_ctl_info.loc[:,'Ingredients'] = df_ctl_info.loc[:,'Ingredients'].apply(lambda x: use_regex(x))

In [26]:
df_ctl_info

,Name,Ingredients
0,American Beauty Cocktail,Brandy Dry Vermouth White Creme De Menthe Orange Juice Grenadine Syrup Red Port
1,Azzuro,ExoticFruit Liqueur Blue Curacao Liqueur Banana Liqueur Passion Fruit Juice Apple Juice Egg
2,Apple Fairy,2 Herbal Liqueur 2 Apple Vodka Apple Soda
3,Black Rose Bacardi,Ice White Rum Cola Lime Juice
4,Blue Latvian,Bourbon Whiskey Raspberry Juice LemonLime Soda
...,...,...
5111,Brown Hen,Vodka tsp Bitters
5112,Bosom Caresser,Crushed Ice Brandy Madeira Triple Sec
5113,Brandy Water Press,Brandy Tonic Water Water
5114,Big Dipper,Dark Rum Brandy Lime Juice Sugar s Orange Liqueur Club Soda


## Clean data for tokenization

In [27]:
vectorizer = TfidfVectorizer(stop_words='english')

In [28]:
word_list = df_ctl_info["Ingredients"].apply(lambda x: ' '.join(x.split())).tolist()

In [29]:
type(word_list[0])

str

In [30]:
def extra_regex(input_text):
    pattern = re.sub(r'\b(\d+(?:\.\d+)?\s*(?:oz|s)\b)(?!\w)', '', input_text)
    return pattern

In [31]:
filtered_list = list(map(extra_regex, word_list))

In [32]:
print(filtered_list)

['Brandy Dry Vermouth White Creme De Menthe Orange Juice Grenadine Syrup Red Port', 'ExoticFruit Liqueur Blue Curacao Liqueur Banana Liqueur Passion Fruit Juice Apple Juice Egg', '2 Herbal Liqueur 2 Apple Vodka Apple Soda', 'Ice White Rum Cola Lime Juice', 'Bourbon Whiskey Raspberry Juice LemonLime Soda', 'White Rum Lemon Juice Superfine Sugar 3 Club Soda', 'Bourbon Whiskey Bourbon Whiskey  Peach Liqueur Sours', 'Raspberry Liqueur Peach Liqueur Lemonade Sours', 'Coffee Liqueur Scotch Whiskey s Soda Water', 'Brandy Orange Liqueur', 'Blue Curacao Liqueur Orange Liqueur Water 1 Oranges 1 Grapefruit 1 Tamarillo', 'Mandarin Vodka Orange Rum Orange Liqueur Peach Schnapps Grenadine Syrup 5 Orange Juice', 'Gin Pepper Soda  Vodka 1 Spiced Rum 1 Triple Sec 1 Chocolate Milk Chocolate Milk s Chocolate Milk Chocolate Milk Energy Soda', 'Rum Apple Juice  Lime Juice Sugar', 'Tequila Apple Brandy s Triple Sec Lemon Juice 3 Maple Syrup', 'Lemon Rum Blue Curacao Liqueur 1 Grenadine Syrup Sours s Soda Wa

In [33]:
def remove_numerical(input_text):
    pattern = re.sub(r'\d+', '', input_text)
    return pattern

In [34]:
filtered_list = list(map(remove_numerical, filtered_list))

In [35]:
filtered_list

['Brandy Dry Vermouth White Creme De Menthe Orange Juice Grenadine Syrup Red Port',
 'ExoticFruit Liqueur Blue Curacao Liqueur Banana Liqueur Passion Fruit Juice Apple Juice Egg',
 ' Herbal Liqueur  Apple Vodka Apple Soda',
 'Ice White Rum Cola Lime Juice',
 'Bourbon Whiskey Raspberry Juice LemonLime Soda',
 'White Rum Lemon Juice Superfine Sugar  Club Soda',
 'Bourbon Whiskey Bourbon Whiskey  Peach Liqueur Sours',
 'Raspberry Liqueur Peach Liqueur Lemonade Sours',
 'Coffee Liqueur Scotch Whiskey s Soda Water',
 'Brandy Orange Liqueur',
 'Blue Curacao Liqueur Orange Liqueur Water  Oranges  Grapefruit  Tamarillo',
 'Mandarin Vodka Orange Rum Orange Liqueur Peach Schnapps Grenadine Syrup  Orange Juice',
 'Gin Pepper Soda  Vodka  Spiced Rum  Triple Sec  Chocolate Milk Chocolate Milk s Chocolate Milk Chocolate Milk Energy Soda',
 'Rum Apple Juice  Lime Juice Sugar',
 'Tequila Apple Brandy s Triple Sec Lemon Juice  Maple Syrup',
 'Lemon Rum Blue Curacao Liqueur  Grenadine Syrup Sours s Soda

In [36]:
index = df_ctl_info["Name"]

In [37]:
vectors = vectorizer.fit_transform(filtered_list).todense()

In [38]:
vocabulary = vectorizer.get_feature_names_out()

In [39]:
token_matrix = pd.DataFrame(vectors, columns=vocabulary, index=index)

In [40]:
pd.set_option('display.max_columns', 507)
token_matrix

,absinthe,advocaat,agave,aged,alcohol,alcoholic,ale,allspice,almond,amaro,amarula,amber,american,amp,angelica,angostura,anis,anise,aperol,apple,apples,apricot,aquavit,arak,bacardi,baileys,banana,bananas,bananastrawberry,bananes,bannanas,bar,base,basil,bbq,beans,bears,beaten,beef,beer,beers,benedictine,berries,berry,birch,bison,bitter,bitters,black,blackberries,blackberry,blackcurrant,blackcurrants,blended,blood,bloodorange,bloody,blue,blueberries,blueberry,boiling,bois,bollinger,bottle,bouillon,bourbon,brandy,bread,breezer,brewed,bristow,brown,bull,burgundy,butter,butterfly,butterscotch,cacao,cachaca,cactus,cake,campari,canadian,candied,candy,cantaloupe,cappuccino,caramel,caraway,cardamom,caribbean,carrot,cassis,caster,cayenne,celery,chai,champagne,chardonnay,chartreuse,cheese,cherries,cherry,chicken,chili,chilli,chips,chocchip,chocolate,chopped,cider,cinnamon,citrus,clam,clamato,clementine,cloudberry,cloves,club,coast,cocktail,cocoa,coconut,coffee,cognac,cola,colada,cold,collins,colour,colouring,comfort,concentrate,condensed,congnac,conserve,cookies,cool,cooler,cordial,coriander,corn,cornstarch,corona,coronita,cottage,cracked,crackers,cranberries,cranberry,cranberryapple,cranberrygrape,cranberryraspberry,cream,crem,creme,crushed,cube,cubes,cucumber,cucumbers,cup,cups,curacao,daiquiri,dark,dashes,demerara,des,didnt,dill,distilled,divided,double,draft,dragonberry,drink,dry,dust,eclair,edible,egg,eggnog,elderflower,energy,evaporated,exoticfruit,extract,eye,fennel,fills,finely,fireball,float,flowers,food,forbidden,fraise,frambois,fresh,freshly,frozen,fruit,garnish,gatorade,gelatin,gilbey,gin,giner,ginger,gingerbread,glow,glycerine,gold,gomme,grain,grape,grapefruit,grapes,grass,grated,green,grenadine,ground,guava,guavaberry,guinness,gummi,halfandhalf,happened,hard,harmonie,hazelnut,hazelnuts,heavy,herbal,honey,horseradish,hot,hpnotiq,ice,iced,icewine,inch,infused,irish,italian,jager,jalapeno,jamaican,jello,jelly,juice,kiwi,kiwistrawberry,koolaid,kryptonite,kumquat,lace,lamb,larger,lavender,layer,leaves,lemon,lemonade,lemonlime,lemons,licorice,light,lime,liqueur,llemonlime,lots,luxardo,lychee,lychees,mace,madeira,malt,mandarin,mango,mangos,maple,maraschino,margarita,marshmallows,mary,mashed,melon,men,menthe,merlot,mexican,mezcal,milk,mineral,mint,mix,mixed,molasses,moscato,mure,muscatel,mustard,nectar,noodle,nutella,nutmeg,nuts,ojen,olive,olives,onion,op,optional,orange,orangebanana,oranges,oregano,oreo,organic,orgeat,original,ounce,oz,papaya,passion,passionfruit,pea,peach,peaches,peachmango,peachs,peanut,peanuts,pear,pearl,pears,pecan,peel,pepper,peppermint,peppers,pernod,peychauds,pickle,pimms,pina,pinch,pineapple,pineapples,pineorangebanana,pink,pisco,pitch,plum,plums,pomegranate,port,powder,powdered,preferred,premix,prosecco,prune,pumpkin,punch,puree,rainbow,raisins,raspberries,raspberry,recipe,recommended,red,redcurrant,rice,rind,rinse,rock,root,rosemary,ruby,rum,rye,sage,salt,sambuca,sarsaparilla,sauce,schanpps,schnapps,scotch,seasonal,sec,serve,serving,shavings,sherbert,sherbet,sherry,shots,shredded,silver,simple,slice,sliced,slices,sloe,smirnoff,soda,soft,soup,sour,sours,southern,soy,sparkling,spice,spiced,spicy,splash,spoon,sprite,squash,squeezed,stalk,starfruits,steak,stick,stout,straw,strawberries,strawberry,strawberrybanana,strawberrykiwi,strega,strips,sugar,superfine,sweet,syrup,tabasco,tajin,tamarillo,tangerine,tea,tennessee,tequila,toffee,tomato,tonic,topping,triple,tropical,tropicalfruit,tropico,try,tsp,tsps,tuaca,twist,unflavored,unflavoured,vanilla,vegetable,vermouth,version,vinegar,viniq,violette,vitamin,voda,vodak,vodka,walnut,water,watermelon,watermeloncherry,wedge,wedges,welch,west,wheel,wheels,whipped,whipping,whiskey,whisky,white,wildberry,wilderberry,wine,withchampagne,withcoffee,withcream,withlemonade,woodruff,woops,worcestershire,work,wormwood,yeast,yellow,yoghurt,yolk
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

## Preprocess data

In [41]:
X = token_matrix.reset_index()
X.head()

,Name,absinthe,advocaat,agave,aged,alcohol,alcoholic,ale,allspice,almond,amaro,amarula,amber,american,amp,angelica,angostura,anis,anise,aperol,apple,apples,apricot,aquavit,arak,bacardi,baileys,banana,bananas,bananastrawberry,bananes,bannanas,bar,base,basil,bbq,beans,bears,beaten,beef,beer,beers,benedictine,berries,berry,birch,bison,bitter,bitters,black,blackberries,blackberry,blackcurrant,blackcurrants,blended,blood,bloodorange,bloody,blue,blueberries,blueberry,boiling,bois,bollinger,bottle,bouillon,bourbon,brandy,bread,breezer,brewed,bristow,brown,bull,burgundy,butter,butterfly,butterscotch,cacao,cachaca,cactus,cake,campari,canadian,candied,candy,cantaloupe,cappuccino,caramel,caraway,cardamom,caribbean,carrot,cassis,caster,cayenne,celery,chai,champagne,chardonnay,chartreuse,cheese,cherries,cherry,chicken,chili,chilli,chips,chocchip,chocolate,chopped,cider,cinnamon,citrus,clam,clamato,clementine,cloudberry,cloves,club,coast,cocktail,cocoa,coconut,coffee,cognac,cola,colada,cold,collins,colour,colouring,comfort,concentrate,condensed,congnac,conserve,cookies,cool,cooler,cordial,coriander,corn,cornstarch,corona,coronita,cottage,cracked,crackers,cranberries,cranberry,cranberryapple,cranberrygrape,cranberryraspberry,cream,crem,creme,crushed,cube,cubes,cucumber,cucumbers,cup,cups,curacao,daiquiri,dark,dashes,demerara,des,didnt,dill,distilled,divided,double,draft,dragonberry,drink,dry,dust,eclair,edible,egg,eggnog,elderflower,energy,evaporated,exoticfruit,extract,eye,fennel,fills,finely,fireball,float,flowers,food,forbidden,fraise,frambois,fresh,freshly,frozen,fruit,garnish,gatorade,gelatin,gilbey,gin,giner,ginger,gingerbread,glow,glycerine,gold,gomme,grain,grape,grapefruit,grapes,grass,grated,green,grenadine,ground,guava,guavaberry,guinness,gummi,halfandhalf,happened,hard,harmonie,hazelnut,hazelnuts,heavy,herbal,honey,horseradish,hot,hpnotiq,ice,iced,icewine,inch,infused,irish,italian,jager,jalapeno,jamaican,jello,jelly,...,kiwistrawberry,koolaid,kryptonite,kumquat,lace,lamb,larger,lavender,layer,leaves,lemon,lemonade,lemonlime,lemons,licorice,light,lime,liqueur,llemonlime,lots,luxardo,lychee,lychees,mace,madeira,malt,mandarin,mango,mangos,maple,maraschino,margarita,marshmallows,mary,mashed,melon,men,menthe,merlot,mexican,mezcal,milk,mineral,mint,mix,mixed,molasses,moscato,mure,muscatel,mustard,nectar,noodle,nutella,nutmeg,nuts,ojen,olive,olives,onion,op,optional,orange,orangebanana,oranges,oregano,oreo,organic,orgeat,original,ounce,oz,papaya,passion,passionfruit,pea,peach,peaches,peachmango,peachs,peanut,peanuts,pear,pearl,pears,pecan,peel,pepper,peppermint,peppers,pernod,peychauds,pickle,pimms,pina,pinch,pineapple,pineapples,pineorangebanana,pink,pisco,pitch,plum,plums,pomegranate,port,powder,powdered,preferred,premix,prosecco,prune,pumpkin,punch,puree,rainbow,raisins,raspberries,raspberry,recipe,recommended,red,redcurrant,rice,rind,rinse,rock,root,rosemary,ruby,rum,rye,sage,salt,sambuca,sarsaparilla,sauce,schanpps,schnapps,scotch,seasonal,sec,serve,serving,shavings,sherbert,sherbet,sherry,shots,shredded,silver,simple,slice,sliced,slices,sloe,smirnoff,soda,soft,soup,sour,sours,southern,soy,sparkling,spice,spiced,spicy,splash,spoon,sprite,squash,squeezed,stalk,starfruits,steak,stick,stout,straw,strawberries,strawberry,strawberrybanana,strawberrykiwi,strega,strips,sugar,superfine,sweet,syrup,tabasco,tajin,tamarillo,tangerine,tea,tennessee,tequila,toffee,tomato,tonic,topping,triple,tropical,tropicalfruit,tropico,try,tsp,tsps,tuaca,twist,unflavored,unflavoured,vanilla,vegetable,vermouth,version,vinegar,viniq,violette,vitamin,voda,vodak,vodka,walnut,water,watermelon,watermeloncherry,wedge,wedges,welch,west,wheel,wheels,whipped,whipping,whiskey,whisky,white,wildberry,wilderberry,wine,withchampagne,withcoffee,withcream,withlemonade,woodruff,woops,worcestershire,work,wormwood,yeast,yellow,yoghurt,yolk
0,American Beauty Cocktail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00

In [42]:
X = X.drop('Name', axis=1)
X.head()

,absinthe,advocaat,agave,aged,alcohol,alcoholic,ale,allspice,almond,amaro,amarula,amber,american,amp,angelica,angostura,anis,anise,aperol,apple,apples,apricot,aquavit,arak,bacardi,baileys,banana,bananas,bananastrawberry,bananes,bannanas,bar,base,basil,bbq,beans,bears,beaten,beef,beer,beers,benedictine,berries,berry,birch,bison,bitter,bitters,black,blackberries,blackberry,blackcurrant,blackcurrants,blended,blood,bloodorange,bloody,blue,blueberries,blueberry,boiling,bois,bollinger,bottle,bouillon,bourbon,brandy,bread,breezer,brewed,bristow,brown,bull,burgundy,butter,butterfly,butterscotch,cacao,cachaca,cactus,cake,campari,canadian,candied,candy,cantaloupe,cappuccino,caramel,caraway,cardamom,caribbean,carrot,cassis,caster,cayenne,celery,chai,champagne,chardonnay,chartreuse,cheese,cherries,cherry,chicken,chili,chilli,chips,chocchip,chocolate,chopped,cider,cinnamon,citrus,clam,clamato,clementine,cloudberry,cloves,club,coast,cocktail,cocoa,coconut,coffee,cognac,cola,colada,cold,collins,colour,colouring,comfort,concentrate,condensed,congnac,conserve,cookies,cool,cooler,cordial,coriander,corn,cornstarch,corona,coronita,cottage,cracked,crackers,cranberries,cranberry,cranberryapple,cranberrygrape,cranberryraspberry,cream,crem,creme,crushed,cube,cubes,cucumber,cucumbers,cup,cups,curacao,daiquiri,dark,dashes,demerara,des,didnt,dill,distilled,divided,double,draft,dragonberry,drink,dry,dust,eclair,edible,egg,eggnog,elderflower,energy,evaporated,exoticfruit,extract,eye,fennel,fills,finely,fireball,float,flowers,food,forbidden,fraise,frambois,fresh,freshly,frozen,fruit,garnish,gatorade,gelatin,gilbey,gin,giner,ginger,gingerbread,glow,glycerine,gold,gomme,grain,grape,grapefruit,grapes,grass,grated,green,grenadine,ground,guava,guavaberry,guinness,gummi,halfandhalf,happened,hard,harmonie,hazelnut,hazelnuts,heavy,herbal,honey,horseradish,hot,hpnotiq,ice,iced,icewine,inch,infused,irish,italian,jager,jalapeno,jamaican,jello,jelly,juice,kiwi,kiwistrawberry,koolaid,kryptonite,kumquat,lace,lamb,larger,lavender,layer,leaves,lemon,lemonade,lemonlime,lemons,licorice,light,lime,liqueur,llemonlime,lots,luxardo,lychee,lychees,mace,madeira,malt,mandarin,mango,mangos,maple,maraschino,margarita,marshmallows,mary,mashed,melon,men,menthe,merlot,mexican,mezcal,milk,mineral,mint,mix,mixed,molasses,moscato,mure,muscatel,mustard,nectar,noodle,nutella,nutmeg,nuts,ojen,olive,olives,onion,op,optional,orange,orangebanana,oranges,oregano,oreo,organic,orgeat,original,ounce,oz,papaya,passion,passionfruit,pea,peach,peaches,peachmango,peachs,peanut,peanuts,pear,pearl,pears,pecan,peel,pepper,peppermint,peppers,pernod,peychauds,pickle,pimms,pina,pinch,pineapple,pineapples,pineorangebanana,pink,pisco,pitch,plum,plums,pomegranate,port,powder,powdered,preferred,premix,prosecco,prune,pumpkin,punch,puree,rainbow,raisins,raspberries,raspberry,recipe,recommended,red,redcurrant,rice,rind,rinse,rock,root,rosemary,ruby,rum,rye,sage,salt,sambuca,sarsaparilla,sauce,schanpps,schnapps,scotch,seasonal,sec,serve,serving,shavings,sherbert,sherbet,sherry,shots,shredded,silver,simple,slice,sliced,slices,sloe,smirnoff,soda,soft,soup,sour,sours,southern,soy,sparkling,spice,spiced,spicy,splash,spoon,sprite,squash,squeezed,stalk,starfruits,steak,stick,stout,straw,strawberries,strawberry,strawberrybanana,strawberrykiwi,strega,strips,sugar,superfine,sweet,syrup,tabasco,tajin,tamarillo,tangerine,tea,tennessee,tequila,toffee,tomato,tonic,topping,triple,tropical,tropicalfruit,tropico,try,tsp,tsps,tuaca,twist,unflavored,unflavoured,vanilla,vegetable,vermouth,version,vinegar,viniq,violette,vitamin,voda,vodak,vodka,walnut,water,watermelon,watermeloncherry,wedge,wedges,welch,west,wheel,wheels,whipped,whipping,whiskey,whisky,white,wildberry,wilderberry,wine,withchampagne,withcoffee,withcream,withlemonade,woodruff,woops,worcestershire,work,wormwood,yeast,yellow,yoghurt,yolk
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.

In [43]:
y = token_matrix.reset_index()
y = y['Name']
y.head()

0    American Beauty Cocktail
1                      Azzuro
2                 Apple Fairy
3          Black Rose Bacardi
4                Blue Latvian
Name: Name, dtype: object

## Split train/test

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train model

In [45]:
model = NearestNeighbors(n_neighbors=3)

model.fit(X_train, X_train)

NearestNeighbors(n_neighbors=3)

_______________________________

## User input preprocessing

In [46]:
input_vectorizer = TfidfVectorizer(stop_words='english')

In [47]:
vectorizer.fit(X)

TfidfVectorizer(stop_words='english')

In [48]:
def preprocess_input(input):
    input = input.lower()
    input = re.sub("[0-9]", "",input)
    input = [input]
    input = vectorizer.transform(input)
    return input

In [49]:
user_input = input('Please enter an ingredient')
print(user_input)
input_vector = preprocess_input(user_input)
print(input_vector)

martini pina colada
  (0, 348)	0.7071067811865476
  (0, 126)	0.7071067811865476


## Finding similar ingredients

In [50]:
distances, indices = model.kneighbors(input_vector, n_neighbors=3)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [51]:
similar_ingredients = y_train.iloc[indices[0]].values

In [52]:
similar_ingredients = y_train.iloc[indices[0]].values

In [55]:
recommended_ing = []
print("Recommended ingredients:")
for ingredient in similar_ingredients:
    recommended_ing.append(ingredient)
print(recommended_ing)

Recommended ingredients:
['Double Bubble', 'Malibu Orange Colada Punch', 'Cayman Sunset']


In [54]:
model.fit(X_train, X_train)  # Pass X_train as the target variable

def get_similar_ingredients(input_vector, model, vectorizer, top_k=3):
    input_vector = vectorizer.transform([input_vector])
    distances, indices = model.kneighbors(input_vector, n_neighbors=top_k)
    similar_ingredients = [vectorizer.inverse_transform(X_train[index])[0] for index in indices.flatten()]
    return similar_ingredients

user_input = input('Please enter an ingredient: ')
similar_ingredients = get_similar_ingredients(user_input, model, vectorizer)
print(similar_ingredients)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


KeyError: 195

## Full cocktail info

In [ ]:
ctl_info = pd.read_csv('./CSVs/cocktail_info.csv', sep=',')

In [ ]:
ctl_info.loc[2116]

# RECOMMENDER RETURNS COCKTAIL NAMES !  
invert X and y to have "mixable ingredients" ?  

Increment 2 : have the recommender also shit out the makable cocktail names + recipe & all other info

______________________________

__________________________________

fter fitting the model with the training data using model.fit(X_train, y_train), the next steps would involve:

* Preprocessing the user input:

Transform the user input into a TF-IDF vector using the same TfidfVectorizer that was used for the training data.
Make sure to preprocess the user input in the same way as the training data (e.g., lowercase, remove punctuation, etc.).  

* Finding similar ingredients:

Transform the preprocessed user input into a feature vector using the trained TfidfVectorizer: input_vector = vectorizer.transform([user_input]).  
  
Use the kneighbors() method of the trained model to find the nearest neighbors based on the input vector:  
distances, indices = model.kneighbors(input_vector, n_neighbors=3).  

The indices array will contain the indices of the nearest neighbors in the training data.  

Retrieve the corresponding ingredient names from the training data using these indices: similar_ingredients = y_train[indices].  

* Displaying the results:

Present the similar_ingredients to the user as the recommended ingredients that are likely to appear together with the user inputted ingredient in multiple cocktails.

___________________________

In [ ]:
user_ingredient = 'vodka'
preprocessed_ingredient = preprocess_ingredient(user_ingredient)

# Compute pairwise similarity between user-inputted ingredient and all ingredients
distances, indices = nbrs.kneighbors(CountVectorizer.transform([preprocessed_ingredient]))

# Select the top k ingredients that are most similar
k = 5  # Number of similar ingredients to retrieve
similar_ingredients = df.iloc[indices[0], :]['Name'].values[:k]

# Print the predicted ingredients
print(similar_ingredients)

In [ ]:
k = 5
model = NearestNeighbors(n_neighbors=k, metric='hamming')  # 'hamming' for binary data

# Fit the data to the KNN model
knn.fit(X)

# Find the k nearest neighbors for each cocktail
distances, indices = knn.kneighbors(X)

# Iterate over the indices to analyze the neighbors
for i, cocktail_indices in enumerate(indices):
    cocktail_ingredients = []
    for idx in cocktail_indices:
        ingredients = ...  # Get the ingredients corresponding to the index
        cocktail_ingredients.extend(ingredients)